In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel

# Load the tokenizer
summarization_tokenizer = AutoTokenizer.from_pretrained("./lora_summarization_model")

# Load the base model
base_model_name = "t5-small"  # Replace with your original base model if different
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)

# Load the LoRA fine-tuned model
lora_model = PeftModel.from_pretrained(base_model, "./lora_summarization_model")

2024-12-03 17:54:08.020927: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-03 17:54:09.139228: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-12-03 17:54:09.139361: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [4]:
from transformers import AutoTokenizer, MBartForConditionalGeneration, AutoModelForSeq2SeqLM
from peft import PeftModel

# Load the translation model and tokenizer
translation_model = MBartForConditionalGeneration.from_pretrained("./translation_model")
translation_tokenizer = AutoTokenizer.from_pretrained("./translation_model")

# Load the summarization model and tokenizer
summarization_tokenizer = AutoTokenizer.from_pretrained("./lora_summarization_model")
base_model_name = "t5-small"  # Replace with your original base model if different
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)
lora_model = PeftModel.from_pretrained(base_model, "./lora_summarization_model")

# Translation function
def translate_text(text):
    prompt = f"Translate the text to English: {text}"
    inputs = translation_tokenizer(prompt, return_tensors="pt", max_length=128, truncation=True)
    outputs = translation_model.generate(**inputs)
    translation = translation_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

def pipeline(input_text):
    
    # Translate text
    translated_text = translate_text(input_text)
    print("Translated text: ",translated_text)
    
    # Summarize translated text
    prompt = f"summarize this text: {translated_text}"
    inputs = summarization_tokenizer(
            prompt,
            return_tensors="pt",
            max_length=512,
            truncation=True
        )    
    # Generate summary
    outputs = lora_model.generate(
        **inputs,
        max_length=150,
        min_length=20,
        num_beams=4,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    summary = summarization_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [6]:
example_text = """واشنطن – مع عودة الكونجرس هذا الأسبوع للقيام بموجة أخيرة من النشاط قبل أن يختتم جلسته، فإنه يواجه موعدًا نهائيًا رئيسيًا في 20 ديسمبر لتجنب إغلاق الحكومة.

يبدو أن الديمقراطيين والجمهوريين مستسلمون لتمرير قرار مستمر، أو CR، من شأنه أن يمول الحكومة مؤقتًا حتى أوائل عام 2025 - على الأرجح في مارس - حيث ينفد الوقت أمامهم للتوصل إلى اتفاق تمويل كامل هذا العام. ولم يتفق الطرفان حتى على مستوى الإنفاق الإجمالي للسنة المالية الجديدة، ناهيك عن كيفية تخصيص الأموال عبر أجزاء من الحكومة.

ألمح زعيم الأغلبية في مجلس الشيوخ تشاك شومر، ديمقراطي من ولاية نيويورك، إلى حتمية مشروع قانون قصير الأجل يوم الاثنين، قائلاً: "نحن بحاجة إلى إبقاء الأحكام المثيرة للخلاف وغير الضرورية خارج أي تمديد للتمويل الحكومي، وإلا فسيكون من الصعب تمرير مشروع قانون". CR في الوقت المناسب.

وبالنسبة للجمهوريين، فإن هذا سيف ذو حدين.

الجانب الإيجابي بالنسبة للجمهوريين في تحديد الموعد النهائي هو أنهم سيكون لديهم المزيد من النفوذ لتشكيل التمويل الحكومي في العام الجديد، مع عودة الرئيس المنتخب دونالد ترامب إلى البيت الأبيض وسيطرة الحزب الجمهوري على مجلس الشيوخ والحفاظ على أغلبية ضيقة في مجلس النواب.

الجانب السلبي الكبير هو أنه سيحدد موعدًا نهائيًا حاسمًا في وقت مبكر من رئاسة ترامب، مما قد يستغرق وقتًا ثمينًا بعيدًا عن تأكيد مرشحيه من خلال مجلس الشيوخ ومن مشروع قانون الحزب الكبير الذي يتطلع إليه الجمهوريون لتمديد تخفيضاته الضريبية وتعزيز الهجرة. وأجندة أمن الحدود.

وأضاف كينيدي أنه من المرجح أن يعلق الكونجرس "30 [مليار دولار] إلى 40 مليار دولار من الإغاثة في حالات الكوارث" إلى الجمهورية التشيكية، بما في ذلك تمويل الولايات التي ضربتها الأعاصير هذا العام. وقال: "لن يكون ذلك كافيا، لكنه سيكون كافيا للبدء"."""

result = pipeline(example_text)
print("\nSummary:", result)

Translated text:  WASHINGTON, DC – With Congress returning this week to make one final move before it closes, it faces a major deadline on December 20 to avoid shutting down the government. Democrats and Republicans seem resigned to pushing through a decision, or CR, that would temporarily fund the government until early 2025 – probably in March – when they run out of time to agree on full funding this year.

Summary: Congress faces a major deadline on December 20 to avoid shutting down the government. Democrats and Republicans seem resigned to pushing through an CR decision that would temporarily fund government until early 2025 - probably in March – when they run out of time to agree on full funding this year.
